<a href="https://colab.research.google.com/github/Jesse-Allister-Kasien-Elliott/Simplex-Algorithm-/blob/master/simplex_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Simplex Algorithm Implementation in Python (for Google Colab)
# --------------------------------------------------------------
# This script implements the Simplex algorithm for solving linear programming problems
# in standard form: maximize c^T x subject to Ax = b, x >= 0.

import numpy as np

def simplex(c, A, b, BAS):
    m, n = A.shape

    # Compute non-basic variables N
    N = []
    for i in range(n):
        if i not in BAS:
            N.append(i)

    # Extract cB and cN
    cB = np.array([c[i] for i in BAS]).reshape(-1, 1)
    cN = np.array([c[i] for i in N]).reshape(-1, 1)

    # Extract AB and AN
    AB = A[:, BAS]
    AN = A[:, N]

    ABinv = np.linalg.inv(AB)
    xB = ABinv @ b
    xN = np.zeros((n - m, 1))

    # Main Simplex loop
    cBarT = cN.T - (cB.T @ ABinv @ AN)

    while np.any(cBarT > 0):
        # Find entering variable j
        entering_indices = np.where(cBarT > 0)[1]
        k = entering_indices[0]
        j = N[k]

        # Determine direction dB
        dB = -ABinv @ A[:, j]

        # Check for unboundedness
        if np.all(dB >= 0):
            print("UNBOUNDED")
            return 1, dB

        # Determine leaving variable
        theta_vec = -xB.flatten() / dB.flatten()
        valid_indices = np.where(dB < 0)[0]
        theta_vec = theta_vec[valid_indices]
        l_index = valid_indices[np.argmin(theta_vec)]
        l = BAS[l_index]

        # Pivot step: update basis
        BAS[l_index] = j
        N[k] = l
        BAS.sort()
        N.sort()

        # Update matrices and vectors
        cB = np.array([c[i] for i in BAS]).reshape(-1, 1)
        cN = np.array([c[i] for i in N]).reshape(-1, 1)
        AB = A[:, BAS]
        AN = A[:, N]
        ABinv = np.linalg.inv(AB)
        xB = ABinv @ b
        xN = np.zeros((n - m, 1))
        cBarT = cN.T - (cB.T @ ABinv @ AN)

    # Construct full solution
    x = np.zeros((n, 1))
    for i, idx in enumerate(BAS):
        x[idx] = xB[i]
    for idx in N:
        x[idx] = 0

    return 0, x

# Example 1 (bounded)
c = np.array([1, 4, 0, 0])
A = np.array([[1, 2, 1, 0],
              [1, 1, 0, 1]])
b = np.array([[2], [4]])
BAS = [0, 3]  # Using 0-based indexing in Python

status, solution = simplex(c, A, b, BAS)

print("Status:", "Optimal" if status == 0 else "Unbounded")
print("Solution:")
print(solution)


Status: Optimal
Solution:
[[0.]
 [1.]
 [0.]
 [3.]]
